In [ ]:
!pip uninstall openai

In [ ]:
!pip install llmx==0.0.15a0
!pip install cohere
!pip install openai
!pip install tiktoken

In [ ]:
import pandas as pd
import os
import json
import time
from openai import OpenAI

In [ ]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [ ]:
def batch_gpt(message_prompt):
    responses = []
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[{"role": "user", "content": message_prompt}]
    )
    r = response.choices[0].message.content
    responses.append(r)

    return r, responses

In [ ]:
def get_mess():
  mess = """
You need to determine whether the existing discourse relations between 2 elementary discourse units is elaboration or not.  Discourse relations between 2 discourse units can be explicit or implicit.
An elaboration relation in discourse refers to a type of connection between two statements or units of text where the second unit provides additional information, details, or clarification about the first one.
The following json is an example of elementary discourse unit pairs that have relation of type elaboration:

"sentences_with_elaboration_relation" : {
	example_1 : [
	"Preparedness and response plans for MERS and poliomyelitis were in place in the 			participating countries",
	"with a high level of technical expertise available to implement them"
	],

	example_2 : [
	"This result revealed an interesting finding",
	"which is that the influence of cognitive age on seniors ' use intention depends on 			seniors ' level of subjective well - being"
	]
}

I will give you a text and 2 elementary discourse units extracted from a text and then i will give you the whole text. In the context of the example discourse units, discourse_unit_1 comes before discourse_unit_2. You should  determine if the type of the existing discourse relation between 2 given example elementary discourse units is elaboration or something else.
Return the result in json format and do not add anny addition explanations to the answer.
example: {"discourse_relation" : "elaboration"}, if the relation is indeed elaboration,
example: {"discourse_relation" : "None"}, if the relation is not elaboration,

      """

  return mess

In [ ]:
dataset = pd.read_csv('/local_path/eng.dep.covdtb_test.rels.csv', sep='\t')
df = pd.DataFrame(columns=['EDU_1', 'EDU_2', 'Real discourse relation', 'ChatGPT discourse relation', 'Second guess'])
unprocessed_data = pd.DataFrame(columns=['unit1_sent', 'unit2_sent', 'doc_id', 'label'])
print(dataset.shape)

(2586, 12)


In [ ]:
dataset.head()

,doc,unit1_toks,unit2_toks,unit1_txt,unit2_txt,s1_toks,s2_toks,unit1_sent,unit2_sent,dir,orig_label,label
0,00c862a6402381a8541bcffe843696e059095300,1-18,19-22,"In this study , we investigated turkey reoviru...",aged 15 days .,1-22,1-22,"In this study , we investigated turkey reoviru...","In this study , we investigated turkey reoviru...",1<2,elaboration,elaboration
1,00c862a6402381a8541bcffe843696e059095300,1-18,23-32,"In this study , we investigated turkey reoviru...",RT - PCR for TReoV detected 3.3 % positive sam...,1-22,23-41,"In this study , we investigated turkey reoviru...",RT - PCR for TReoV detected 3.3 % positive sam...,1<2,findings,findings
2,00c862a6402381a8541bcffe843696e059095300,23-32,33-41,RT - PCR for TReoV detected 3.3 % positive sam...,and TReoV was successfully isolated in Vero ce...,23-41,23-41,RT - PCR for TReoV detected 3.3 % positive sam...,RT - PCR for TReoV detected 3.3 % positive sam...,1<2,joint,joint
3,00c862a6402381a8541bcffe843696e059095300,23-32,42-58,RT - PCR for TReoV detected 3.3 % positive sam...,Histological analysis of positive bursa of Fab...,23-41,42-58,RT - PCR for TReoV detected 3.3 % positive sam...,Histological analysis of positive bursa of Fab...,1<2,elaboration,elaboration
4,00c862a6402381a8541bcffe843696e059095300,42-58,59-76,Histological analysis of positive bursa of Fab...,"The number of CD8 ? , CD4 ? and IgM ? cells wa...",42-58,59-76,Histological analysis of positive bursa of Fab...,"The number of CD8 ? , CD4 ? and IgM ? cells wa...",1<2,elaboration,elaboration


In [ ]:
dataset.shape

(2586, 12)

In [ ]:
data_with_texts = pd.read_csv('/local_path/eng_dep_covdtb_texts_test.csv')
data_with_texts.head()

,Document ID,Document Abstract
0,00c862a6402381a8541bcffe843696e059095300,"In this study , we investigated turkey reoviru..."
1,0146a0b36f80ebcb868c7e16fb60c1a9d2afbf83,This is an extensive review on epiphytic plant...
2,0232f18e7bbf3192e9e34da1c0abe2a20b2ba1e4,"Herein , the LASSBio Chemical Library is prese..."
3,09bdbea1700a3826acc33484ed51e2fa8c23f047,Data - centric models of COVID-19 have been tr...
4,0b362bc7e9278b56c2740ab7bc10e91c036f835a,We study seasonal epidemic spreading in a susc...


In [ ]:
message_part1 = get_mess()

for i in range(0, len(dataset)):
  print()
  doc_id = dataset.iloc[i]['doc']
  sentence_1 = dataset.iloc[i]['unit1_txt'] 
  sentence_2 = dataset.iloc[i]['unit2_txt']  
  real_label = dataset.iloc[i]['label']

  row = data_with_texts[data_with_texts['Document ID'] == doc_id]
  document_text = row['Document Abstract'].values[0]

  message_part3 = {
      "text" : document_text
  }

  message_part4 = {
        "discourse_unit_1" : sentence_1,
        "discourse_unit_2" : sentence_2
  }

  prompt_message = message_part1 + '\n' + '\nexample_text = ' + str(message_part3) + '\n' + '\nexample_discourse_units = ' + str(message_part4)
  print(str(i) + " " + str(message_part4))
  second_guess = ""
  try:
    result, response = batch_gpt(prompt_message)
    result = result.replace('json', '')
    result = result.replace('`', '')
    result = result.strip()
    json_result = json.loads(result)
    gpt_relation = json_result['discourse_relation']

  except Exception as e:
    print("ERROR")
    print(e)
    unprocessed_data.loc[len(unprocessed_data)] = {'unit1_txt': sentence_1, 'unit2_txt': sentence_2, 'doc_id': doc_id, 'label': real_label}
  else:
    new_row = {'EDU_1': sentence_1, 'EDU_2': sentence_2, 'Real discourse relation': real_label, 'ChatGPT discourse relation': gpt_relation, 'Second guess': second_guess}
    df.loc[len(df)] = new_row

    if i % 300 == 0:
      time.sleep(10)


In [ ]:
df.head()

,EDU_1,EDU_2,Real discourse relation,ChatGPT discourse relation,Second guess
0,"In this study , we investigated turkey reoviru...",aged 15 days .,elaboration,None,
1,"In this study , we investigated turkey reoviru...",RT - PCR for TReoV detected 3.3 % positive sam...,findings,None,
2,RT - PCR for TReoV detected 3.3 % positive sam...,and TReoV was successfully isolated in Vero ce...,joint,elaboration,
3,RT - PCR for TReoV detected 3.3 % positive sam...,Histological analysis of positive bursa of Fab...,elaboration,None,
4,Histological analysis of positive bursa of Fab...,"The number of CD8 ? , CD4 ? and IgM ? cells wa...",elaboration,elaboration,


In [ ]:
unprocessed_data.to_csv('gpt4_eng_dep_covdtb_test_unprocessed_prompt_messagges_only_elaboration.csv', index=False)
df.to_csv('gpt4_eng_dep_covdtb_test_results_only_elaboration.csv', index=False)

In [ ]:
df['ChatGPT discourse relation'].unique()

array(['None', 'elaboration', 'contrast', 'Contrast'], dtype=object)

In [ ]:
df['Real discourse relation'].unique()

array(['elaboration', 'findings', 'joint', 'attribution', 'manner-means',
       'enablement', 'cause-result', 'background', 'comparison',
       'textual-organization', 'temporal', 'condition'], dtype=object)